In [10]:
from nsga2.estimator import NSGAIIRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/lexicase_paper/d_airfoil.txt', sep=',')

# DEAP interface requires X and y to be numpy arrays, not pandas dataframes
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

estimator = NSGAIIRegressor(**{
    'pop_size'        : 40, 
    'max_gen'         : 50,
    'max_depth'       : 7,  # 8
    'max_size'        : 2**7, # 75
    'objectives'      : ['error', 'size'],
    'initialization'  : 'uniform',
    'pick_criteria'   : 'error', # error, MCDM
    'validation_size' : 0.33,
    'simplify'        : False,
    
    # Either you use smart variation (just 1 cx and 1 mutation)
    'smart_variation' : True,
    'warm_up'         : False,

    # Or you use mabs (4 mutations)
    'use_mab'         : False,
    'use_context'     : False,

    'simplification_method' : 'bottom_up',
    'simplification_tolerance' : 1e-0,
    'verbosity'       : 1,
    'survival'       : 'tournament'
}).fit(X_train, y_train)

hashtable will have dimensions 256 x 755
starting to index. 755, 0
starting to index. 755, 1
starting to index. 755, 2
starting to index. 755, 3
starting to index. 755, 4
starting to index. 755, 5
initialized 6 keys
gen	evals	best_size	best_error	n_simplifications	n_new_hashes	avg train error	avg train size	avg val error	avg val size	med train error	med train size	med val error	med val size	std train error	std train size	std val error	std val size	min train error	min train size	min val error	min val size	max train error	max train size	max val error	max val size
0  	40   	11       	-6780.99  	0                	0           	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            
1  	40   	14       	-5182.69  	0                	0           	               	  

KeyboardInterrupt: 

In [2]:
# Should not give erros even without mabs
pd.DataFrame(estimator.variator.mab.pull_history).iloc[:10]

,t,arm,reward,update,delta_error,gen
0,0,lsh_mutate,0.0,0,"[-12114.37558725102, 1.0]",1
1,1,cx,1.0,0,"[17747107.21896585, -0.0]",1
2,2,cx,0.0,0,"[-7763.0767679877345, 5.0]",1
3,3,cx,0.0,0,"[-inf, -15.0]",1
4,4,subtree,1.0,0,"[inf, 23.0]",1
5,5,cx,0.0,0,"[-2.3593461700997756e+36, -2.0]",1
6,6,lsh_mutate,0.0,0,"[-5.127681756891983e+121, 4.0]",1
7,7,subtree,1.0,0,"[8.518542178326309e+122, -6.0]",1
8,8,lsh_mutate,0.0,0,"[-84.20625815388667, 4.0]",1
9,9,subtree,1.0,0,"[2509.0196817434753, 2.0]",1


In [3]:
pd.DataFrame(estimator.variator.mab.pull_history)['arm'].value_counts().sort_values()

subtree       647
cx            656
lsh_mutate    657
Name: arm, dtype: int64

In [4]:
pd.DataFrame(estimator.variator.mab.pull_history).groupby('arm')['reward'].value_counts().sort_values()

arm         reward
subtree     0.0        80
cx          0.0        98
lsh_mutate  0.0       148
subtree     0.5       172
cx          1.0       239
lsh_mutate  1.0       247
            0.5       262
cx          0.5       319
subtree     1.0       395
Name: reward, dtype: int64

In [5]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

model      = str(estimator.best_estimator_).replace("ARG", "x_")
size       = len(estimator.best_estimator_)
complexity = size
depth      = estimator.best_estimator_.height

print(model)
print(size)
print(complexity)
print(depth)

for metric, fn, (data_X, data_y) in [
    ('train_r2',  r2_score, (X_train, y_train)),
    ('test_r2',   r2_score, (X_test,  y_test )),
    ('train_mse', mse,      (X_train, y_train)),
    ('test_mse',  mse,      (X_test,  y_test )),
]:
    score = np.nan
    try:
        score = fn(estimator.predict(data_X), data_y)
        print(f"{metric} : {score}")
    except ValueError:
        print(f"(Failed to calculate {metric}")

for ind in estimator.archive_:
    print(ind.fitness, ind)

add(127.74665684499173, subtract(arctan(exp(add3(tan(x_3), log(x_1), cos(x_0)))), subtract(mul3(add3(add4(x_4, x_1, x_4, x_2), add3(x_4, x_1, arctan(x_2)), -22.868135522264367), x_4, mul3(-2.387433438196368e-07, add4(mul4(x_4, x_1, x_3, x_1), 77.80218554038836, x_4, x_2), square(multiply(x_2, x_0)))), multiply(add(x_4, -0.2983368215314719), mul3(add3(482.85727920928866, x_2, x_0), x_4, maximum(subtract(x_2, x_1), add(0.399133067185192, x_4)))))))
59
59
7
train_r2 : -0.04414261768584704
test_r2 : -0.19137299447335288
train_mse : 23.538422533317206
test_mse : 26.67147487330208
(22.31802058201198, 59.0) add(127.74665684499173, subtract(arctan(exp(add3(tan(ARG3), log(ARG1), cos(ARG0)))), subtract(mul3(add3(add4(ARG4, ARG1, ARG4, ARG2), add3(ARG4, ARG1, arctan(ARG2)), -22.868135522264367), ARG4, mul3(-2.387433438196368e-07, add4(mul4(ARG4, ARG1, ARG3, ARG1), 77.80218554038836, ARG4, ARG2), square(multiply(ARG2, ARG0)))), multiply(add(ARG4, -0.2983368215314719), mul3(add3(482.85727920928866,

<string>:1: RuntimeWarning: divide by zero encountered in log
<string>:1: RuntimeWarning: divide by zero encountered in log
<string>:1: RuntimeWarning: divide by zero encountered in log
<string>:1: RuntimeWarning: divide by zero encountered in log


In [6]:
if False:
    print( len(list(estimator.simplifier.pop_hash.keys())) )

    n_keys =  len(list(estimator.simplifier.pop_hash.keys()))

    for key in list(estimator.simplifier.pop_hash.keys())[:n_keys]:
        print(key)
        for ind in estimator.simplifier.pop_hash[key]:
            print(" -", ind)

In [7]:
if True:
    print( len(list(estimator.variator.variator_.pop_hash.keys())) )

    n_keys =  len(list(estimator.variator.variator_.pop_hash.keys()))

    for key in list(estimator.variator.variator_.pop_hash.keys())[:n_keys]:
        print(key)
        for ind in estimator.variator.variator_.pop_hash[key]:
            print(" -", ind)

1936
0
 - 1.0
1
 - ARG0
 - add4(ARG1, -86.41245922255882, ARG3, ARG0)
 - subtract(ARG0, -91.07568296978022)
 - maximum(ARG0, ARG2)
 - maximum(add(ARG0, ARG3), add4(ARG1, ARG3, ARG3, ARG3))
 - add(ARG0, ARG3)
 - add4(ARG4, ARG0, 153.6967244193987, ARG0)
 - add4(arctan(ARG0), mul3(ARG4, ARG4, ARG0), ARG0, ARG4)
 - add3(ARG1, ARG0, ARG2)
 - add(subtract(-99.86580759118016, ARG4), add(ARG4, ARG0))
 - add(ARG4, ARG0)
 - add4(ARG1, ARG1, ARG0, ARG2)
 - absolute(ARG0)
 - add3(ARG4, ARG1, ARG0)
 - maximum(sqrtabs(add4(ARG3, ARG2, ARG2, ARG0)), ARG0)
 - add4(ARG3, ARG2, ARG2, ARG0)
 - add4(ARG4, log(add3(tan(ARG0), add3(ARG3, ARG4, 27.982134946920183), square(ARG1))), 153.66373167612593, ARG0)
 - minimum(ARG0, ARG0)
 - cdiv(ARG0, 62.60490916005952)
 - maximum(ARG1, ARG0)
 - add4(ARG3, ARG4, ARG0, ARG0)
 - add3(ARG0, ARG0, ARG3)
 - add3(cos(sqrtabs(mul3(1.0232115473243226, ARG0, ARG0))), add4(tan(add4(ARG3, 66.41047359877555, ARG3, ARG2)), multiply(sin(ARG4), sqrtabs(-78.53629966254476)), exp(ab

In [9]:
display(df.describe())

A = np.maximum(df['x0'], df['x4']+29.657).values
B = df['x4'].values

print(np.std(A) * (A - np.mean(A))[:5])
print(np.std(B) * (B - np.mean(B))[:5])

,x0,x1,x2,x3,x4,label
count,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000,1503.000000
mean,2886.380572,6.782302,0.136548,50.860745,0.011140,124.835943
std,3152.573137,5.918128,0.093541,15.572784,0.013150,6.898657
min,200.000000,0.000000,0.025400,31.700000,0.000401,103.380000
25%,800.000000,2.000000,0.050800,39.600000,0.002535,120.191000
50%,1600.000000,5.400000,0.101600,39.600000,0.004957,125.721000
75%,4000.000000,9.900000,0.228600,71.300000,0.015576,129.995500
max,20000.000000,22.200000,0.304800,71.300000,0.058411,140.987000


[-6575278.86832892 -5944974.02788168 -5157092.97732263 -4054059.50653996
 -2793449.82564547]
[-0.00011143 -0.00011143 -0.00011143 -0.00011143 -0.00011143]
